In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import numpy as np
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.grid_search import RandomizedSearchCV
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn import metrics
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plts
import time 
import random

/Users/stuartgeman/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/stuartgeman/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
data=pd.read_csv('Income and Bike Id/july_med_bike_id.csv')

In [3]:
#Categorize Borough
borough_total=pd.read_csv('Borough_Code.csv')
num_borough=len(borough_total)
for i in range(num_borough):
    data=data.replace(borough_total.iloc[i,1],borough_total.iloc[i,0])

#Categorize Neighborhood
entries=len(data)
nbhd_total=pd.read_csv('Neighborhood_Code.csv')
num_nbhd=len(nbhd_total)
for i in range(num_nbhd):
    data=data.replace(nbhd_total.iloc[i,1],nbhd_total.iloc[i,0])

#Categorize UserType
user_total=pd.read_csv('User_Code.csv')
num_user=len(user_total)
for i in range(num_user):
    data=data.replace(user_total.iloc[i,1],user_total.iloc[i,0])

#Categorize Weather Description
weather_total=pd.read_csv('Weather_Code.csv')
num_weather=len(weather_total)
for i in range(num_weather):
    data=data.replace(weather_total.iloc[i,1],weather_total.iloc[i,0])
#Drop unwanted columns
data=data.drop(labels=drop_names,axis=1)

#Fill na's with 0's for income
data['start median income']=data['start median income'].fillna(value=0)

In [4]:
#Fill na's with 0's for income
data['start median income']=data['start median income'].fillna(value=0)

In [11]:
#Train on unfiltered dataset
msk = np.random.rand(len(data)) < 0.8

train = data[msk]
y = np.array(train['end neighborhood'])
X = train.drop(['end neighborhood'], axis =1)

test = data[~msk]
y_test = np.array(test['end neighborhood'])
X_test = test.drop('end neighborhood', axis =1)

In [12]:
randtree = RandomForestClassifier(max_depth=None)
#params = {'max_depth': [2, 20,], 'n_estimators':[10, 50, 100, 250, 500], 'max_features':[8, 9]}
#RFgrid = RandomizedSearchCV(estimator=randtree, param_distributions=params, cv=5, scoring = 'accuracy')
#RFgrid.fit(trainnp,predictnp)
randtree.fit(X, y)
#RandTreeOut=model_to_csv(RFgrid,name='RandomForest.csv')
#RFimportances=randtree.feature_importances_

#Random Tree on unfiltered data
metrics.accuracy_score(y_test,randtree.predict(X_test))

0.38308778600040738

In [15]:
#Random Forest
#Train on morning weekdays
morning_data = data[(data['time'] >= 600) & (data['time'] <=900) & (data['day'] <=4) &
                   (data['gender'] ==2) & (data['birth year'] >= 1988.0)]
# morning_data.drop('usertype', axis = 1)
msk = np.random.rand(len(morning_data)) < 0.8

train = morning_data[msk]
y = np.array(train['end neighborhood'])
X = train.drop(['end neighborhood'], axis =1)

test = morning_data[~msk]
y_test = np.array(test['end neighborhood'])
X_test = test.drop(['end neighborhood'], axis =1)

#Random Fit on morning weekdays 
randtree = RandomForestClassifier(n_estimators = 160,max_depth=None)
randtree.fit(X,y)
#Random Tree results on morning weekdays 
metrics.accuracy_score(y_test,randtree.predict(X_test))

0.62166356300434511

In [17]:
#insight
importances = randtree.feature_importances_
columns = data.columns

importance_frame = pd.DataFrame(importances, X.columns)
importance_frame

,0
tripduration,0.214694
start station id,0.087708
start station latitude,0.114485
start station longitude,0.102207
usertype,0.000212
birth year,0.108038
gender,0.000000
time,0.048930
humidity,0.063308
tempf,0.060598


In [18]:
#Decision Tree
#Train on morning weekdays
morning_data = data[(data['time'] >= 600) & (data['time'] <=900) & (data['day'] <=4) &
                   (data['gender'] ==2) & (data['birth year'] >= 1988.0)]
# morning_data.drop('usertype', axis = 1)
msk = np.random.rand(len(morning_data)) < 0.8

train = morning_data[msk]
y = np.array(train['end neighborhood'])
X = train.drop(['end neighborhood'], axis =1)

test = morning_data[~msk]
y_test = np.array(test['end neighborhood'])
X_test = test.drop(['end neighborhood'], axis =1)

#Decision Tree Fit on morning weekdays 
DTC = DecisionTreeClassifier(max_depth=60, min_samples_leaf = 5, min_samples_split = 2)
DTC = DTC.fit(X,y)
#Decision Tree results on morning weekdays 
metrics.accuracy_score(y_test,DTC.predict(X_test))

0.51761918862896061

In [19]:
#KNN classifier
#Train on morning weekdays
morning_data = data[(data['time'] >= 600) & (data['time'] <=900) & (data['day'] <=4) &
                   (data['gender'] ==2) & (data['birth year'] >= 1988.0)]
# morning_data.drop('usertype', axis = 1)
msk = np.random.rand(len(morning_data)) < 0.8

train = morning_data[msk]
y = np.array(train['end neighborhood'])
X = train.drop(['end neighborhood'], axis =1)

test = morning_data[~msk]
y_test = np.array(test['end neighborhood'])
X_test = test.drop(['end neighborhood'], axis =1)

#Create KNN classifier
neigh = KNeighborsClassifier(n_neighbors = 5, leaf_size = 80, weights = 'distance')
neigh.fit(X, y) 
metrics.accuracy_score(y_test,neigh.predict(X_test))

0.48169014084507045

In [21]:
#Voting Classifier RF and DT
#Train on morning weekdays
morning_data = data[(data['time'] >= 600) & (data['time'] <=900) & (data['day'] <=4) &
                   (data['gender'] ==2) & (data['birth year'] >= 1988.0)]
# morning_data.drop('usertype', axis = 1)
msk = np.random.rand(len(morning_data)) < 0.8

train = morning_data[msk]
y = np.array(train['end neighborhood'])
X = train.drop(['end neighborhood'], axis =1)

test = morning_data[~msk]
y_test = np.array(test['end neighborhood'])
X_test = test.drop(['end neighborhood'], axis =1)


# create classifers for voting classifer
RFC = RandomForestClassifier(n_estimators = 160) 
DTC = DecisionTreeClassifier(max_depth=60, min_samples_leaf = 5, min_samples_split = 2)
neigh = KNeighborsClassifier(n_neighbors = 5, leaf_size = 80, weights = 'distance')


#Create Voting Classifier
eclf = VotingClassifier(estimators=[('RFC', RFC), ("DT", DTC), ('neigh', neigh)], voting = 'soft', weights = [2,1,1])
eclf.fit(X, y)
metrics.accuracy_score(y_test,eclf.predict(X_test))

0.61224489795918369

In [22]:
#RF GRID TREE
#Train on morning weekdays
morning_data = data[(data['time'] >= 600) & (data['time'] <=900) & (data['day'] <=4) &
                   (data['gender'] ==2)]
# morning_data.drop('usertype', axis = 1)
msk = np.random.rand(len(morning_data)) < 0.8

train = morning_data[msk]
y = np.array(train['end neighborhood'])
X = train.drop(['end neighborhood'], axis =1)

test = morning_data[~msk]
y_test = np.array(test['end neighborhood'])
X_test = test.drop(['end neighborhood'], axis =1)

#Grid tree with params selected below on morning weekdays
params = {'max_depth': [2, 20, 40], 'n_estimators':[10, 50, 100, 250, 500], 'max_features':[8, 9]}
RFgrid = RandomizedSearchCV(estimator=randtree, param_distributions=params, cv=5, scoring = 'accuracy')
RFgrid.fit(X,y)
#RandTreeOut=model_to_csv(RFgrid,name='RandomForest.csv')
#RFimportances=randtree.feature_importances_
#Grid tree results on morning weekdays
metrics.accuracy_score(y_test,RFgrid.predict(X_test))

/Users/stuartgeman/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


0.62225085910652922

In [26]:
#Train on Evening Weekdays
evening_data = data[(data['time'] >= 1800) & (data['time'] <=2100) & (data['day'] <=4)]
msk = np.random.rand(len(evening_data)) < 0.8

train = evening_data[msk]
y = np.array(train['end neighborhood'])
X = train.drop(['end neighborhood'], axis =1)

test = evening_data[~msk]
y_test = np.array(test['end neighborhood'])
X_test = test.drop(['end neighborhood'], axis =1)

In [27]:
randtree = RandomForestClassifier(n_estimators = 160)
#params = {'max_depth': [2, 20,], 'n_estimators':[10, 50, 100, 250, 500], 'max_features':[8, 9]}
#RFgrid = RandomizedSearchCV(estimator=randtree, param_distributions=params, cv=5, scoring = 'accuracy')
#RFgrid.fit(trainnp,predictnp)
randtree.fit(X, y)
#RandTreeOut=model_to_csv(RFgrid,name='RandomForest.csv')
#RFimportances=randtree.feature_importances_
metrics.accuracy_score(y_test,randtree.predict(X_test))

0.36015641513422109

In [ ]:
#Train on Evening Weekends

wknd_evening_data = data[(data['time'] >= 1800) & (data['time'] <=2100) & (data['day'] >4)]
msk = np.random.rand(len(wknd_evening_data)) < 0.8

train = wknd_evening_data[msk]
y = np.array(train['end neighborhood'])
X = train.drop(['end neighborhood'], axis =1)

test = wknd_evening_data[~msk]
y_test = np.array(test['end neighborhood'])
X_test = test.drop(['end neighborhood'], axis =1)

In [ ]:
randtree = RandomForestClassifier(max_depth=None)
#params = {'max_depth': [2, 20,], 'n_estimators':[10, 50, 100, 250, 500], 'max_features':[8, 9]}
#RFgrid = RandomizedSearchCV(estimator=randtree, param_distributions=params, cv=5, scoring = 'accuracy')
#RFgrid.fit(trainnp,predictnp)
randtree.fit(X, y)
#RandTreeOut=model_to_csv(RFgrid,name='RandomForest.csv')
#RFimportances=randtree.feature_importances_
metrics.accuracy_score(y_test,randtree.predict(X_test))

In [ ]:
imp=randtree.feature_importances_
features=X.columns
pd.DataFrame(imp,features)

In [28]:
# Simply guess the most common end neighborhood for the given start stations on morning weekday results
morning_data = data[(data['time'] >= 600) & (data['time'] <=900) & (data['day'] <=4) &
                      (data['gender'] == 2) & (data['birth year'] >= 1988.0)]

#stations: a dataframe with start station id and end neighborhood
stations = pd.DataFrame(morning_data['start station id'])
stations['end neighborhood'] = morning_data['end neighborhood']

#the set of station ids
station_set = set(morning_data['start station id'])

# most common neighborhood destination for each station 
station_id = []
frequent_hood = []
for i in station_set:
    x = stations[(stations['start station id'] ==i)]['end neighborhood']
    y = x.value_counts()
    station_id.append(i)
    frequent_hood.append(y.index[0])
    
#Create dataframe with above lists (station_ids and most coommon neighborhood visited)
station_destination = pd.DataFrame(station_id)
station_destination['frequent hood'] = frequent_hood
station_destination.columns = ['start station id', 'frequent hood']

#Add frequent hood entries to corresponding columns in original data frame
poop = morning_data
result = pd.merge(poop, station_destination, on='start station id', how='outer')

#Result if we just geuss the most frequen neighborhood corresponding to each station
metrics.accuracy_score(result['end neighborhood'],result['frequent hood'])

0.37588827782240625

In [29]:
# Simply guess the most common end neighborhood for the given start stations
# For unfiltered demographic data

#stations: a dataframe with start station id and end neighborhood
stations = pd.DataFrame(data['start station id'])
stations['end neighborhood'] = data['end neighborhood']

#the set of station ids
station_set = set(data['start station id'])

# most common neighborhood destination for each station 
station_id = []
frequent_hood = []
for i in station_set:
    x = stations[(stations['start station id'] ==i)]['end neighborhood']
    y = x.value_counts()
    station_id.append(i)
    frequent_hood.append(y.index[0])
    
#Create dataframe with above lists (station_ids and most coommon neighborhood visited)
station_destination = pd.DataFrame(station_id)
station_destination['frequent hood'] = frequent_hood
station_destination.columns = ['start station id', 'frequent hood']

#Add frequent hood entries to corresponding columns in original data frame
poop = data
result = pd.merge(poop, station_destination, on='start station id', how='outer')

#Result if we just geuss the most frequen neighborhood corresponding to each station
metrics.accuracy_score(result['end neighborhood'],result['frequent hood'])

0.26628400850993661

In [18]:
#This time we will test on the three 'summerish' months to see how we do.
July = pd.read_csv('Income and Bike Id/july_med_bike_id.csv')
August = pd.read_csv('Income and Bike Id/august_med_bike_id.csv')
June = pd.read_csv('Income and Bike Id/june_med_bike_id.csv')

In [19]:
result = pd.concat([June, July, August])

In [ ]:
#Columns to drop from the DataFrame
drop_names=['tripduration','start station name', 'end station name','end station id','end borough', 'end station latitude', 'end station longitude','start_time', 
            'stop_time', 'date','start point', 'end point','start median income','end median income', 'bikeid']
#Categorize Borough
num_borough=len(set(result['start borough']))
borough=list(set(result['start borough']))
borough_code=[i for i in range(num_borough)]
borough_total=pd.DataFrame(borough,borough_code)
for i in range(num_borough):
    result=result.replace(borough_total[[0]].iloc[i][0],i)
    
#Categorize Neighborhood
entries=len(result)
num_nbhd=len(set(result['end neighborhood']))
nbhds=list(set(result['end neighborhood']))
nbhd_code=[i for i in range(num_nbhd)]
nbhd_total=pd.DataFrame(nbhds,nbhd_code)
for i in range(num_nbhd):
    result=result.replace(nbhd_total[[0]].iloc[i][0],i)

#Categorize UserType
num_user=len(set(result['usertype']))
user_type=list(set(result['usertype']))
user_code=[i for i in range(num_user)]
user_total=pd.DataFrame(user_type,user_code)
for i in range(num_user):
    result=result.replace(user_total[[0]].iloc[i][0],i)

#Categorize Weather Description
num_weather=len(set(result['weatherdesc']))
weather_type=list(set(result['weatherdesc']))
weather_code=[i for i in range(num_weather)]
weather_total=pd.DataFrame(weather_type,weather_code)
for i in range(num_weather):
    result=result.replace(weather_total[[0]].iloc[i][0],i)
    
#Drop unwanted columns
result=result.drop(labels=drop_names,axis=1)

#Fill na's with 0's for income
result['start median income'] = result['start median income'].fillna(value=0)
result.head()

,start station id,start station latitude,start station longitude,usertype,birth year,gender,time,humidity,tempf,weatherdesc,windspeedmiles,day,start neighborhood,start borough,end neighborhood,bikeid
0,380,40.734011,-74.002939,0,1972.0,1,0,75,76,15,4,2,27,0,21,19859
1,3092,40.719009,-73.958525,0,1967.0,1,0,75,76,15,4,2,0,1,0,16233
2,449,40.764618,-73.987895,0,1989.0,1,0,75,76,15,4,2,21,0,4,22397
3,522,40.757148,-73.972078,0,1991.0,1,0,75,76,15,4,2,10,0,28,16231
4,335,40.729039,-73.994046,0,1989.0,1,0,75,76,15,4,2,27,0,27,15400


In [ ]:
#Train on unfiltered dataset
msk = np.random.rand(len(result)) < 0.8

train = result[msk]
y = np.array(train['end neighborhood'])
X = train.drop(['end neighborhood'], axis =1)

test = result[~msk]
y_test = np.array(test['end neighborhood'])
X_test = test.drop('end neighborhood', axis =1)

#Fit randtree 
randtree = RandomForestClassifier(n_estimators=160)
randtree.fit(X,y)

#Random Tree results
metrics.accuracy_score(y_test,randtree.predict(X_test))

In [ ]:
#Random Forest
#Train on filtered dataset: morning weekdays
morning_data = result[(result['time'] >= 600) & (result['time'] <= 900) & (result['day'] <= 4) 
                      & (result['gender'] == 2) & (result['birth year'] >= 1988.0)]

msk = np.random.rand(len(morning_data)) < 0.8

train = morning_data[msk]
y = np.array(train['end neighborhood'])
X = train.drop(['end neighborhood'], axis =1)

test = morning_data[~msk]
y_test = np.array(test['end neighborhood'])
X_test = test.drop('end neighborhood', axis =1)

#Fit randtree 
randtree = RandomForestClassifier(n_estimators = 160)
randtree.fit(X,y)

#Random Forest Tree results
metrics.accuracy_score(y_test,randtree.predict(X_test))

In [ ]:
#KNN classifier
#Train on morning weekdays
morning_data = result[(result['time'] >= 600) & (result['time'] <= 900) & (result['day'] <= 4) 
                      & (result['gender'] == 2) & (result['birth year'] >= 1988.0)]

# morning_data.drop('usertype', axis = 1)
msk = np.random.rand(len(morning_data)) < 0.8

train = morning_data[msk]
y = np.array(train['end neighborhood'])
X = train.drop(['end neighborhood'], axis =1)

test = morning_data[~msk]
y_test = np.array(test['end neighborhood'])
X_test = test.drop(['end neighborhood'], axis =1)

#Create KNN classifier
neigh = KNeighborsClassifier(n_neighbors = 5, leaf_size = 80, weights = 'distance')
neigh.fit(X, y) 
metrics.accuracy_score(y_test,neigh.predict(X_test))

In [ ]:
#Decision Tree
#Train on morning weekdays
morning_data = result[(result['time'] >= 600) & (result['time'] <= 900) & (result['day'] <= 4) 
                      & (result['gender'] == 2) & (result['birth year'] >= 1988.0)]
# morning_data.drop('usertype', axis = 1)
msk = np.random.rand(len(morning_data)) < 0.8

train = morning_data[msk]
y = np.array(train['end neighborhood'])
X = train.drop(['end neighborhood'], axis =1)

test = morning_data[~msk]
y_test = np.array(test['end neighborhood'])
X_test = test.drop(['end neighborhood'], axis =1)

#Decision Tree Fit on morning weekdays 
DTC = DecisionTreeClassifier(max_depth=60, min_samples_leaf = 5, min_samples_split = 2)
DTC = DTC.fit(X,y)
#Decision Tree results on morning weekdays 
metrics.accuracy_score(y_test,DTC.predict(X_test))

In [137]:
#Voting Classifier RF and DT and KNN
#Train on morning weekdays
morning_data = result[(result['time'] >= 600) & (result['time'] <= 900) & (result['day'] <= 4) 
                      & (result['gender'] == 2) & (result['birth year'] >= 1988.0)]
# morning_data.drop('usertype', axis = 1)
msk = np.random.rand(len(morning_data)) < 0.8

train = morning_data[msk]
y = np.array(train['end neighborhood'])
X = train.drop(['end neighborhood'], axis =1)

test = morning_data[~msk]
y_test = np.array(test['end neighborhood'])
X_test = test.drop(['end neighborhood'], axis =1)


# create classifers for voting classifer
RFC = RandomForestClassifier(n_estimators = 160) 
DTC = DecisionTreeClassifier(max_depth=60, min_samples_leaf = 5, min_samples_split = 2)
# neigh = KNeighborsClassifier(n_neighbors = 5, leaf_size = 80, weights = 'distance')


#Create Voting Classifier
eclf = VotingClassifier(estimators=[('RFC', RFC), ("DT", DTC)], voting = 'soft', weights = [2,1.5])
eclf.fit(X, y)
metrics.accuracy_score(y_test,eclf.predict(X_test))

0.65596721871265151

In [ ]:
#Train on filtered dataset: morning weekdays
evening_data = result[(result['time'] >= 1500) & (result['time'] <= 1800) & (result['day'] <= 4)]

msk = np.random.rand(len(evening_data)) < 0.8

train = evening_data[msk]
y = np.array(train['end neighborhood'])
X = train.drop(['end neighborhood'], axis =1)

test = evening_data[~msk]
y_test = np.array(test['end neighborhood'])
X_test = test.drop('end neighborhood', axis =1)

#Fit randtree 
randtree = RandomForestClassifier(max_depth=None)
randtree.fit(X,y)

#Random Tree results
metrics.accuracy_score(y_test,randtree.predict(X_test))

In [ ]:
#This time we will test on the three 'Fallish' months to see how we do.
September = pd.read_csv('Income and Bike Id/september_med_bike_id.csv')
October = pd.read_csv('Income and Bike Id/october_med_bike_id.csv')
November = pd.read_csv('Income and Bike Id/november_med_bike_id.csv')
result = pd.concat([September, October, November])

In [ ]:
#Columns to drop from the DataFrame
drop_names=['tripduration','start station name', 'end station name','end station id','end borough', 'end station latitude', 'end station longitude','start_time', 
            'stop_time', 'date','start point', 'end point','start median income','end median income', 'bikeid']
#Categorize Borough
num_borough=len(set(result['start borough']))
borough=list(set(result['start borough']))
borough_code=[i for i in range(num_borough)]
borough_total=pd.DataFrame(borough,borough_code)
for i in range(num_borough):
    result=result.replace(borough_total[[0]].iloc[i][0],i)
    
#Categorize Neighborhood
entries=len(result)
num_nbhd=len(set(result['end neighborhood']))
nbhds=list(set(result['end neighborhood']))
nbhd_code=[i for i in range(num_nbhd)]
nbhd_total=pd.DataFrame(nbhds,nbhd_code)
for i in range(num_nbhd):
    result=result.replace(nbhd_total[[0]].iloc[i][0],i)

#Categorize UserType
num_user=len(set(result['usertype']))
user_type=list(set(result['usertype']))
user_code=[i for i in range(num_user)]
user_total=pd.DataFrame(user_type,user_code)
for i in range(num_user):
    result=result.replace(user_total[[0]].iloc[i][0],i)

#Categorize Weather Description
num_weather=len(set(result['weatherdesc']))
weather_type=list(set(result['weatherdesc']))
weather_code=[i for i in range(num_weather)]
weather_total=pd.DataFrame(weather_type,weather_code)
for i in range(num_weather):
    result=result.replace(weather_total[[0]].iloc[i][0],i)
    
#Drop unwanted columns
result=result.drop(labels=drop_names,axis=1)

#Fill na's with 0's for income
result['start median income']=result['start median income'].fillna(value=0)
result.head()

In [ ]:
#Random Forest
#Train on filtered dataset: morning weekdays
morning_data = result[(result['time'] >= 600) & (result['time'] <= 900) & (result['day'] <= 4) 
                      & (result['gender'] == 2) & (result['birth year'] >= 1988.0)]

msk = np.random.rand(len(morning_data)) < 0.8

train = morning_data[msk]
y = np.array(train['end neighborhood'])
X = train.drop(['end neighborhood'], axis =1)

test = morning_data[~msk]
y_test = np.array(test['end neighborhood'])
X_test = test.drop('end neighborhood', axis =1)

#Fit randtree 
randtree = RandomForestClassifier(n_estimators = 160)
randtree.fit(X,y)

#Random Forest Tree results
metrics.accuracy_score(y_test,randtree.predict(X_test))

In [ ]:
#This time we will test on the three 'Winterish' months to see how we do.
December = pd.read_csv('Income and Bike Id/december_med_bike_id.csv')
January = pd.read_csv('Income and Bike Id/january_med_bike_id.csv')
February = pd.read_csv('Income and Bike Id/february_med_bike_id.csv')
result = pd.concat([December, January, February])

In [ ]:
#Columns to drop from the DataFrame
drop_names=['tripduration','start station name', 'end station name','end station id','end borough', 'end station latitude', 'end station longitude','start_time', 
            'stop_time', 'date','start point', 'end point','start median income','end median income', 'bikeid']
#Categorize Borough
num_borough=len(set(result['start borough']))
borough=list(set(result['start borough']))
borough_code=[i for i in range(num_borough)]
borough_total=pd.DataFrame(borough,borough_code)
for i in range(num_borough):
    result=result.replace(borough_total[[0]].iloc[i][0],i)
    
#Categorize Neighborhood
entries=len(result)
num_nbhd=len(set(result['end neighborhood']))
nbhds=list(set(result['end neighborhood']))
nbhd_code=[i for i in range(num_nbhd)]
nbhd_total=pd.DataFrame(nbhds,nbhd_code)
for i in range(num_nbhd):
    result=result.replace(nbhd_total[[0]].iloc[i][0],i)

#Categorize UserType
num_user=len(set(result['usertype']))
user_type=list(set(result['usertype']))
user_code=[i for i in range(num_user)]
user_total=pd.DataFrame(user_type,user_code)
for i in range(num_user):
    result=result.replace(user_total[[0]].iloc[i][0],i)

#Categorize Weather Description
num_weather=len(set(result['weatherdesc']))
weather_type=list(set(result['weatherdesc']))
weather_code=[i for i in range(num_weather)]
weather_total=pd.DataFrame(weather_type,weather_code)
for i in range(num_weather):
    result=result.replace(weather_total[[0]].iloc[i][0],i)
    
#Drop unwanted columns
result=result.drop(labels=drop_names,axis=1)

#Fill na's with 0's for income
result['start median income']=result['start median income'].fillna(value=0)
result.head()

In [ ]:
#Random Forest
#Train on filtered dataset: morning weekdays
morning_data = result[(result['time'] >= 600) & (result['time'] <= 900) & (result['day'] <= 4) 
                      & (result['gender'] == 2) & (result['birth year'] >= 1988.0)]

msk = np.random.rand(len(morning_data)) < 0.8

train = morning_data[msk]
y = np.array(train['end neighborhood'])
X = train.drop(['end neighborhood'], axis =1)

test = morning_data[~msk]
y_test = np.array(test['end neighborhood'])
X_test = test.drop('end neighborhood', axis =1)

#Fit randtree 
randtree = RandomForestClassifier(n_estimators = 160)
randtree.fit(X,y)

#Random Forest Tree results
metrics.accuracy_score(y_test,randtree.predict(X_test))

In [ ]:
#This time we will test on the three 'Springish' months to see how we do.
March = pd.read_csv('Income and Bike Id/march_med_bike_id.csv')
April = pd.read_csv('Income and Bike Id/april_med_bike_id.csv')
May = pd.read_csv('Income and Bike Id/may_med_bike_id.csv')
result = pd.concat([March, April, May])

In [ ]:
#Columns to drop from the DataFrame
drop_names=['tripduration','start station name', 'end station name','end station id','end borough', 'end station latitude', 'end station longitude','start_time', 
            'stop_time', 'date','start point', 'end point','start median income','end median income', 'bikeid']
#Categorize Borough
num_borough=len(set(result['start borough']))
borough=list(set(result['start borough']))
borough_code=[i for i in range(num_borough)]
borough_total=pd.DataFrame(borough,borough_code)
for i in range(num_borough):
    result=result.replace(borough_total[[0]].iloc[i][0],i)
    
#Categorize Neighborhood
entries=len(result)
num_nbhd=len(set(result['end neighborhood']))
nbhds=list(set(result['end neighborhood']))
nbhd_code=[i for i in range(num_nbhd)]
nbhd_total=pd.DataFrame(nbhds,nbhd_code)
for i in range(num_nbhd):
    result=result.replace(nbhd_total[[0]].iloc[i][0],i)

#Categorize UserType
num_user=len(set(result['usertype']))
user_type=list(set(result['usertype']))
user_code=[i for i in range(num_user)]
user_total=pd.DataFrame(user_type,user_code)
for i in range(num_user):
    result=result.replace(user_total[[0]].iloc[i][0],i)

#Categorize Weather Description
num_weather=len(set(result['weatherdesc']))
weather_type=list(set(result['weatherdesc']))
weather_code=[i for i in range(num_weather)]
weather_total=pd.DataFrame(weather_type,weather_code)
for i in range(num_weather):
    result=result.replace(weather_total[[0]].iloc[i][0],i)
    
#Drop unwanted columns
result=result.drop(labels=drop_names,axis=1)

#Fill na's with 0's for income
result['start median income']=result['start median income'].fillna(value=0)
result.head()

In [ ]:
#Random Forest
#Train on filtered dataset: morning weekdays
morning_data = result[(result['time'] >= 600) & (result['time'] <= 900) & (result['day'] <= 4) 
                      & (result['gender'] == 2) & (result['birth year'] >= 1988.0)]

msk = np.random.rand(len(morning_data)) < 0.8

train = morning_data[msk]
y = np.array(train['end neighborhood'])
X = train.drop(['end neighborhood'], axis =1)

test = morning_data[~msk]
y_test = np.array(test['end neighborhood'])
X_test = test.drop('end neighborhood', axis =1)

#Fit randtree 
randtree = RandomForestClassifier(n_estimators = 160)
randtree.fit(X,y)

#Random Forest Tree results
metrics.accuracy_score(y_test,randtree.predict(X_test))